In [1]:
# Import pandas and read the data pertaining to transactions into a dataframe
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/My Drive/Colab Notebooks

/content/drive/My Drive/Colab Notebooks


In [4]:
fraud_df = pd.read_csv("FinancialFraudDB.csv", index_col=None)

In [5]:
card_replacement_cost = 5
customer_freeze_cost = 3

In [6]:
# Define a cost matrix corresponding to the figure
import numpy as np
cost_matrix = np.zeros((len(fraud_df.index),4))
cost_matrix[:, 0] = customer_freeze_cost *np.ones(len(fraud_df.index))
cost_matrix[:, 1] = fraud_df["Amount"].values
cost_matrix[:, 2] = card_replacement_cost *np.ones(len(fraud_df.index))

In [7]:
#Create labels and feature matrices
y = fraud_df.pop("Class").values
x = fraud_df.values

In [8]:
# Create a train-test split
from sklearn.model_selection import train_test_split
sets = train_test_split(x, y, cost_matrix, test_size=0.25, random_state = 11)
x_train, x_test,y_train,y_test, cost_matrix_train, cost_matrix_test = sets

In [9]:
# Import the decision tree, fit it to the training data and then predict on the testing set
from sklearn import tree
y_pred_test_dt = tree.DecisionTreeClassifier().fit(x_train,y_train).predict(x_test)

In [10]:
!pip install costcla

     |████████████████████████████████| 4.0MB 6.4MB/s 
  Created wheel for pyea: filename=pyea-0.2-cp37-none-any.whl size=6017 sha256=ea1864bad8a19c517844a4db32465b4dda1ae1771d5712a653efc4be3cfa3463
  Stored in directory: /root/.cache/pip/wheels/3c/bf/6e/06f8b0ed23cbc1fe956f8b6756535cb36f0b68c2f0f2342178
Successfully built pyea


In [11]:
# Import the cost-sensitive decision tree, fit it to the training data and then predict on the testing set
from costcla.models import CostSensitiveDecisionTreeClassifier
y_pred_test_csdt = CostSensitiveDecisionTreeClassifier().fit(x_train, y_train, cost_matrix_train).predict(x_test)

In [12]:
# Calculate the savings score of the 2 models
from costcla.metrics import savings_score

print(savings_score(y_test, y_pred_test_dt, cost_matrix_test))
print(savings_score(y_test, y_pred_test_csdt, cost_matrix_test))

0.589451281204447
0.6251576934715533




1.   Step 1: Load the data

1.   Step 2: Set an administrative cost based on the expected cost of replacing a credit card. I did estimate the business cost of freezing a customer's banking operations until all transactions are verified

1.   Step 3. I define the cost matrix based on the parameters which I have defined and it takes into account the administrative cost of replacing a credit card, business interruption from freezing a customer and so on.

1.   Step 4 and 5. Split our data into train-test set

2.   Step 6: Instantiate a simple classifier, train it and then use it to predict on the testing set

2.   Step 7: Utilize the cost-sensitive random forest model from the costcla library to do the same

2.   Step 8: Utilize the savings_score function from costcla to calculate the savings cost of using y_pred on y_true with a cost matrix. The higher the number the larger the cost savings



